In [1]:
import queries
import csv
import functions as fx
import pandas as pd
import core_constants as cc
import sqlite3 as sql

## What does this do?
> This script merges all of the linked data into a single table.  This used to be handled through a bunch of views but those broke the database UI so I needed to move those out of there

> When thinking about linked players, think left to right: 247 - Rivals - NFL Draft - All Conference - NCAA - All American.  If a player has a 247 record then we start there and join it out with every other data souce recorded link available.  But some records are unique to the later sources - so we want to combine each originating source with each recorded link available to the right of it.  

> You end with 6 different dataframes containing all of the unique sourced players with any or none available links.  

In [2]:
# Last year of freshman recruits you want to return
recruitMaxYear = 2016
# First year of NFL Draft results, All Conference or All American awards you want to return
nflMinYear = 2005
acMinYear = 2004
aaMinYear = 2004

In [3]:
query = queries.getLinks247(recruitMaxYear, nflMinYear, acMinYear, aaMinYear)
links247 = fx.connDBAndReturnDF(query)

In [4]:
query = queries.getLinksRivals(recruitMaxYear, nflMinYear, acMinYear, aaMinYear)
linksRivals = fx.connDBAndReturnDF(query)

In [5]:
query = queries.getLinksNFL(nflMinYear, acMinYear, aaMinYear)
linksNFL = fx.connDBAndReturnDF(query)

In [6]:
query = queries.getLinksAC(acMinYear, aaMinYear)
linksAC = fx.connDBAndReturnDF(query)

In [7]:
query = queries.getLinksNCAA(aaMinYear)
linksNCAA = fx.connDBAndReturnDF(query)

In [8]:
query = queries.getLinksAA(aaMinYear)
linksAA = fx.connDBAndReturnDF(query)

In [9]:
dataframes = [links247, linksRivals, linksNFL, linksAC, linksNCAA, linksAA]
all_links = pd.concat(dataframes)

## Write to the Database
> This wipes and re-writes the rows to the Linked Players table.  It removes the index field and uses the default sqlite3 rowid

In [10]:
conn = sql.connect(cc.databaseName)
all_links.to_sql('All_LinkedPlayers', conn, if_exists='replace', index=False)

71045